# Amazon - Baseline

### Pruebas con los distintos clasificadores para obtener los valores de transfer loss sin realizar adaptacion entre dominios

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#clasificadores
from utils.clasificacion import *

#otros
import os
import numpy as np
import pandas as pd
import itertools

#variables para guardar los resultados
tipo = pruebas[0]
dataset_name = datasets[0]

In [3]:
print tipo
print dataset_name
print data_path

baseline
amazon
data


# Pruebas con el dataset Amazon (3000 Dimensiones)

In [4]:
dims = 3000

In [5]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [6]:
i = 1

for tgt in domains:
    print "Entrenando dominio %d de %d: %s" % (i, len(domains), tgt)
    X_tr = labeled[tgt]['X_tr'][: , :dims].todense()
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][: , :dims].todense()
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    ruta_modelo = os.path.join(models_path, dataset_name, "indomain", "%d_%s.pkl" % (dims, tgt))

    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(ruta_modelo, X_tr, y_tr)
    
    print "Score: %.3f" % svc.score(X_ts, y_ts)
    i = i+1
    
print "\nTarea terminada."

Entrenando dominio 1 de 4: electronics
Creando mejor modelo.
Guardando mejor modelo.
Score: 0.889
Entrenando dominio 2 de 4: dvd
Creando mejor modelo.
Guardando mejor modelo.
Score: 0.842
Entrenando dominio 3 de 4: kitchen
Creando mejor modelo.
Guardando mejor modelo.
Score: 0.914
Entrenando dominio 4 de 4: books
Creando mejor modelo.
Guardando mejor modelo.
Score: 0.848

Tarea terminada.


In [7]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

# por cada par posible para adaptar
for src, tgt in pairs:
    print "Tarea %d de %d" % (i+1, tareas)
    
    #baseline in-domain error
    #e_b(T,T)
    #entrenado en dominio tgt y probado en dominio tgt
    X_tr = labeled[tgt]['X_tr'][: , :dims].todense()
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][: , :dims].todense()
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)

    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tr, y_tr)
    b_error = 1-svc.score(X_ts, y_ts)

    #transfer error
    #entrenado en dominio src y probado en dominio tgt sin adaptar
    X_tr = labeled[src]['X_tr'][: , :dims].todense()
    y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][: , :dims].todense()
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    model_name = "%d_%s_%s.pkl" % (dims, src, tgt)
    model_path = os.path.join(models_path, dataset_name, tipo, model_name)
    
    svc2 = load_best_score(model_path, X_tr, y_tr)
    t_error = 1-svc2.score(X_ts, y_ts)


    # transfer loss t
    # t_error - b_error
    t_loss = t_error - b_error

    tarea = src[0]+'->'+tgt[0]
    df.loc[i] = ['Baseline',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
    i+=1
    
print "\nPruebas completadas."

Tarea 1 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 2 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 3 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 4 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 5 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 6 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 7 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 8 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 9 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 10 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 11 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 12 de 12
Cargando modelo existente.

In [8]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,Baseline,e->d,electronics,dvd,15.762894,27.473187,11.710293
1,Baseline,e->k,electronics,kitchen,8.587715,10.322758,1.735043
2,Baseline,e->b,electronics,books,15.157879,28.493212,13.335333
3,Baseline,d->e,dvd,electronics,11.127778,25.263132,14.135353
4,Baseline,d->k,dvd,kitchen,8.587715,18.082952,9.495237
5,Baseline,d->b,dvd,books,15.157879,25.808145,10.650266
6,Baseline,k->e,kitchen,electronics,11.127778,12.625316,1.497537
7,Baseline,k->d,kitchen,dvd,15.762894,26.113153,10.350259
8,Baseline,k->b,kitchen,books,15.157879,30.570764,15.412885
9,Baseline,b->e,books,electronics,11.127778,19.085477,7.957699


In [9]:
new_scores_path1 = os.path.join(scores_path,dataset_name, tipo, "me1_%d.csv" % (dims))
new_scores_path2 = os.path.join(scores_path,dataset_name, tipo, "me2_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path1
print "Guardando en %s" % new_scores_path2

df.to_csv(new_scores_path1, columns=df.columns)
df.to_csv(new_scores_path2, columns=df.columns)
print "Resultados guardados."

Guardando en scores/amazon/baseline/me1_3000.csv
Guardando en scores/amazon/baseline/me2_3000.csv
Resultados guardados.


# Pruebas con el dataset Amazon (1000 Dimensiones)

In [10]:
dims = 1000

In [11]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [31]:
i = 1

for tgt in domains:
    print "Entrenando dominio %d de %d: %s" % (i, len(domains), tgt)
    X_tr = labeled[tgt]['X_tr'][: , :dims].todense()
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][: , :dims].todense()
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    ruta_modelo = os.path.join(models_path, dataset_name, "indomain", "%d_%s.pkl" % (dims, tgt))

    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(ruta_modelo, X_tr, y_tr)
    
    print "Score: %.3f" % svc.score(X_ts, y_ts)
    i = i+1
    
print "\nTarea terminada."

Entrenando dominio 1 de 4: electronics
Cargando modelo existente.
Score: 0.836
Entrenando dominio 2 de 4: dvd
Creando mejor modelo.
Guardando mejor modelo.
Score: 0.831
Entrenando dominio 3 de 4: kitchen
Creando mejor modelo.
Guardando mejor modelo.
Score: 0.838
Entrenando dominio 4 de 4: books
Creando mejor modelo.
Guardando mejor modelo.
Score: 0.763

Tarea terminada.


In [34]:
df = pd.DataFrame(columns=dataframe_columns)


i=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

# por cada par posible para adaptar
for src, tgt in pairs:
    print "Tarea %d de %d" % (i+1, tareas)
    
    #baseline in-domain error
    #e_b(T,T)
    #entrenado en dominio tgt y probado en dominio tgt
    X_tr = labeled[tgt]['X_tr'][: , :dims].todense()
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][: , :dims].todense()
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)


    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tr, y_tr)
    b_error = 1-svc.score(X_ts, y_ts)


    #transfer error
    #entrenado en dominio src y probado en dominio tgt sin adaptar
    X_tr = labeled[src]['X_tr'][: , :dims].todense()
    y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][: , :dims].todense()
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    model_name = "%d_%s_%s.pkl" % (dims, src, tgt)
    model_path = os.path.join(models_path, dataset_name, tipo, model_name)

    svc2 = load_best_score(model_path, X_tr, y_tr)
    t_error = 1-svc2.score(X_ts, y_ts)


    # transfer loss t
    # t_error - b_error
    t_loss = t_error - b_error

    tarea = src[0]+'->'+tgt[0]
    df.loc[i] = ['Baseline',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
    i+=1
    
print "\nPruebas completadas."

Tarea 1 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 2 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 3 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 4 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 5 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 6 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 7 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 8 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 9 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 10 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 11 de 12
Cargando modelo existente.
Creando mejor modelo.
Guardando mejor modelo.
Tarea 12 de 12
Cargando modelo existente.

In [35]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,Baseline,e->d,electronics,dvd,16.900423,26.193155,9.292732
1,Baseline,e->k,electronics,kitchen,16.215405,16.582915,0.367509
2,Baseline,e->b,electronics,books,23.710593,30.493262,6.782670
3,Baseline,d->e,dvd,electronics,16.397910,27.040676,10.642766
4,Baseline,d->k,dvd,kitchen,16.215405,27.133178,10.917773
5,Baseline,d->b,dvd,books,23.710593,25.893147,2.182555
6,Baseline,k->e,kitchen,electronics,16.397910,19.720493,3.322583
7,Baseline,k->d,kitchen,dvd,16.900423,27.080677,10.180255
8,Baseline,k->b,kitchen,books,23.710593,30.798270,7.087677
9,Baseline,b->e,books,electronics,16.397910,26.590665,10.192755


In [36]:
new_scores_path1 = os.path.join(scores_path,dataset_name, tipo, "me1_%d.csv" % (dims))
new_scores_path2 = os.path.join(scores_path,dataset_name, tipo, "me2_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path1
print "Guardando en %s" % new_scores_path2

df.to_csv(new_scores_path1, columns=df.columns)
df.to_csv(new_scores_path2, columns=df.columns)
print "Resultados guardados."

Guardando en scores/amazon/baseline/me1_1000.csv
Guardando en scores/amazon/baseline/me2_1000.csv
Resultados guardados.
